In [1]:
import json
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
import collections
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import  preprocess_input, decode_predictions
from tensorflow.keras.models import Model

In [2]:


# Read the files word_to_idx.pkl and idx_to_word.pkl to get the mappings between word and index
word_to_index = {}
with open ("data/textFiles/word_to_idx.pkl", 'rb') as file:
    word_to_index = pd.read_pickle(file, compression=None)

index_to_word = {}
with open ("data/textFiles/idx_to_word.pkl", 'rb') as file:
    index_to_word = pd.read_pickle(file, compression=None)



print("Loading the model...")
model = load_model('model_checkpoints/model_20.h5')

modelvgg = VGG16(include_top=True,weights=None)
modelvgg.load_weights('/data/disk2/stu_yue/vgg16_weights_tf_dim_ordering_tf_kernels.h5')
modelvgg.layers.pop()
model_new = Model(inputs=modelvgg.inputs, outputs=modelvgg.layers[-2].output)

Loading the model...


In [3]:
# Generate Captions for a random image
# Using Greedy Search Algorithm

def predict_caption(photo):

    inp_text = "startseq"

    #max_len = 80 which is maximum length of caption
    for i in range(80):
        sequence = [word_to_index[w] for w in inp_text.split() if w in word_to_index]
        sequence = pad_sequences([sequence], maxlen=80, padding='post')

        ypred = model.predict([photo, sequence])
        ypred = ypred.argmax()
        word = index_to_word[ypred]

        inp_text += (' ' + word)

        if word == 'endseq':
            break

    final_caption = inp_text.split()[1:-1]
    final_caption = ' '.join(final_caption)
    return final_caption

In [4]:
def preprocess_image (img):
    img = image.load_img(img, target_size=(224, 224))
    img = image.img_to_array(img)

    # Convert 3D tensor to a 4D tendor
    img = np.expand_dims(img, axis=0)

    #Normalize image accoring to ResNet50 requirement
    img = preprocess_input(img)

    return img


In [5]:
# A wrapper function, which inputs an image and returns its encoding (feature vector)
def encode_image (img):
    img = preprocess_image(img)

    feature_vector = model_new.predict(img)
    # feature_vector = feature_vector.reshape((-1,))
    return feature_vector


In [6]:
def runModel(img_name):
    #img_name = input("enter the image name to generate:\t")

#    print("Encoding the image ...")
    photo = encode_image(img_name).reshape((1, 4096))



#    print("Running model to generate the caption...")
    caption = predict_caption(photo)

#    img_data = plt.imread(img_name)
#    plt.imshow(img_data)
#    plt.axis("off")

    #plt.show()
#    print(caption)
    return caption

In [7]:
def readFile (path):
    with open(path, encoding="utf8") as file:
        data = file.read()
    return data;

In [8]:
data = readFile ("data/textFiles/30k_captions.txt")

# Split the data into each line, to get a list of captions
captions = data.split('\n')
# Remove the last line since it is blank
captions = captions[:-1]

In [9]:
content = {}
references={}
for line in captions:
    imageID, caption = line.split('\t')

    imageID = imageID.split('.')[0]

    # If the imageID doesn't exist in the dictionary, create a blank entry
    if content.get(imageID) is None:
        content[imageID] = []
        references[imageID] = []

    # Append the current caption to the list of the corresponding image
    content[imageID].append(caption)
    references[imageID].append(caption.split())

In [10]:
data_val = readFile ("data/textFiles/flickr30k_val.txt")
imageID_val=[]
imageID_val = data_val.split('.jpg\n')
imageID_val=imageID_val[:-1]

In [12]:
from nltk.translate.bleu_score import corpus_bleu
from time import time


generated_captions=['a man in a blue shirt is sitting on a lawn bench in a field'.split()]
scores=[]
good=[]
bad=[]
start_time = time()

for imageID in imageID_val:
    generated_captions=runModel('/data/disk2/stu_yue/flickr/flickr30k-images/flickr30k-images/'+imageID+'.jpg')
    generated_captions=[generated_captions.split()]
    #print('BLEU-1 Score on test data: {:.3f}'.format(corpus_bleu([references[imageID]], generated_captions, weights=(1,0,0,0)) * 100))
    score=corpus_bleu([references[imageID]], generated_captions, weights=(1,0,0,0)) * 100
    print('BLEU-1 Score on test data '+imageID+': {:.3f}'.format(score))
    scores.append(score)
    if (score>50):
        good.append(imageID)
    if (score<30):
        bad.append(imageID)
end_time = time()
print("Total time taken:", end_time-start_time, "sec")


BLEU-1 Score on test data 1018148011: 30.769
BLEU-1 Score on test data 1029450589: 46.667
BLEU-1 Score on test data 1029737941: 38.095
BLEU-1 Score on test data 103205630: 38.462
BLEU-1 Score on test data 10350842: 53.846
BLEU-1 Score on test data 1054620089: 53.846
BLEU-1 Score on test data 1056873310: 42.737
BLEU-1 Score on test data 1072439304: 26.316
BLEU-1 Score on test data 1073444492: 40.000
BLEU-1 Score on test data 1092437557: 38.462
BLEU-1 Score on test data 109671650: 46.154
BLEU-1 Score on test data 109738763: 23.077
BLEU-1 Score on test data 109982467: 21.368
BLEU-1 Score on test data 110269384: 28.491
BLEU-1 Score on test data 1107507919: 45.000
BLEU-1 Score on test data 112243673: 23.077
BLEU-1 Score on test data 1124448967: 45.000
BLEU-1 Score on test data 1126137905: 20.000
BLEU-1 Score on test data 1128874064: 38.095
BLEU-1 Score on test data 1130017585: 38.462
BLEU-1 Score on test data 1141643526: 50.000
BLEU-1 Score on test data 1141718391: 38.462
BLEU-1 Score on te

BLEU-1 Score on test data 241347803: 42.737
BLEU-1 Score on test data 2417341107: 53.846
BLEU-1 Score on test data 2417623030: 21.053
BLEU-1 Score on test data 2420730259: 57.895
BLEU-1 Score on test data 2430018178: 36.842
BLEU-1 Score on test data 2434710077: 53.846
BLEU-1 Score on test data 2439942035: 27.778
BLEU-1 Score on test data 2445654384: 45.000
BLEU-1 Score on test data 2463067409: 28.491
BLEU-1 Score on test data 2473956981: 42.105
BLEU-1 Score on test data 2474483208: 46.154
BLEU-1 Score on test data 2474785004: 49.894
BLEU-1 Score on test data 24759839: 35.000
BLEU-1 Score on test data 2479180530: 33.333
BLEU-1 Score on test data 2481490320: 38.462
BLEU-1 Score on test data 2488783398: 31.417
BLEU-1 Score on test data 2490365757: 30.769
BLEU-1 Score on test data 2493782805: 45.000
BLEU-1 Score on test data 2494088238: 61.538
BLEU-1 Score on test data 2500785647: 35.000
BLEU-1 Score on test data 2502975562: 81.818
BLEU-1 Score on test data 2504683345: 15.385
BLEU-1 Score 

BLEU-1 Score on test data 3226541300: 7.692
BLEU-1 Score on test data 3228517564: 46.667
BLEU-1 Score on test data 3230602641: 46.154
BLEU-1 Score on test data 3237799755: 38.462
BLEU-1 Score on test data 3239891140: 38.380
BLEU-1 Score on test data 3245266444: 34.428
BLEU-1 Score on test data 3251234434: 32.977
BLEU-1 Score on test data 3259222980: 37.898
BLEU-1 Score on test data 3267720211: 50.000
BLEU-1 Score on test data 3268191118: 30.769
BLEU-1 Score on test data 3281225560: 31.669
BLEU-1 Score on test data 3283021089: 35.614
BLEU-1 Score on test data 3288839246: 53.846
BLEU-1 Score on test data 3288928346: 47.368
BLEU-1 Score on test data 3292277400: 27.273
BLEU-1 Score on test data 3294336227: 30.000
BLEU-1 Score on test data 3298378599: 38.095
BLEU-1 Score on test data 3298547199: 30.769
BLEU-1 Score on test data 3300643106: 38.462
BLEU-1 Score on test data 3304763517: 61.538
BLEU-1 Score on test data 3306464579: 40.000
BLEU-1 Score on test data 3310851569: 38.462
BLEU-1 Scor

BLEU-1 Score on test data 3867308040: 45.000
BLEU-1 Score on test data 3868284371: 40.000
BLEU-1 Score on test data 3871177631: 53.846
BLEU-1 Score on test data 3897459678: 38.462
BLEU-1 Score on test data 3899250752: 20.000
BLEU-1 Score on test data 39024981: 35.000
BLEU-1 Score on test data 390921095: 38.462
BLEU-1 Score on test data 391723162: 20.000
BLEU-1 Score on test data 3925836925: 50.000
BLEU-1 Score on test data 3927396708: 36.643
BLEU-1 Score on test data 3929379436: 42.105
BLEU-1 Score on test data 3944169186: 30.000
BLEU-1 Score on test data 3948003394: 46.154
BLEU-1 Score on test data 3970617963: 40.000
BLEU-1 Score on test data 3970645807: 28.491
BLEU-1 Score on test data 3972836203: 35.000
BLEU-1 Score on test data 3977684660: 61.111
BLEU-1 Score on test data 397815951: 20.000
BLEU-1 Score on test data 3987140887: 37.898
BLEU-1 Score on test data 398946421: 33.930
BLEU-1 Score on test data 3996889070: 42.737
BLEU-1 Score on test data 4017326833: 29.172
BLEU-1 Score on 

BLEU-1 Score on test data 4751932433: 38.462
BLEU-1 Score on test data 4760475302: 35.614
BLEU-1 Score on test data 4762117658: 55.000
BLEU-1 Score on test data 4775810124: 36.842
BLEU-1 Score on test data 4775815811: 46.154
BLEU-1 Score on test data 477607006: 46.154
BLEU-1 Score on test data 4783176272: 44.371
BLEU-1 Score on test data 4785056855: 38.462
BLEU-1 Score on test data 4785666083: 40.000
BLEU-1 Score on test data 4786187822: 37.898
BLEU-1 Score on test data 4788507034: 46.154
BLEU-1 Score on test data 4793039083: 42.857
BLEU-1 Score on test data 4793562214: 44.444
BLEU-1 Score on test data 4794844936: 10.000
BLEU-1 Score on test data 4795708965: 61.538
BLEU-1 Score on test data 479807115: 33.333
BLEU-1 Score on test data 4799239390: 57.895
BLEU-1 Score on test data 4799652268: 55.000
BLEU-1 Score on test data 4800150389: 28.424
BLEU-1 Score on test data 4802006392: 35.000
BLEU-1 Score on test data 4803941821: 46.154
BLEU-1 Score on test data 481054596: 46.154
BLEU-1 Score 

BLEU-1 Score on test data 6437497301: 40.000
BLEU-1 Score on test data 6454057197: 30.000
BLEU-1 Score on test data 6457288631: 30.000
BLEU-1 Score on test data 6491222037: 40.000
BLEU-1 Score on test data 6502476865: 21.368
BLEU-1 Score on test data 6524893225: 38.462
BLEU-1 Score on test data 6532234215: 28.537
BLEU-1 Score on test data 6533934303: 10.526
BLEU-1 Score on test data 6563237641: 38.095
BLEU-1 Score on test data 6575547265: 19.025
BLEU-1 Score on test data 6618942119: 35.000
BLEU-1 Score on test data 664341930: 46.154
BLEU-1 Score on test data 664470170: 25.000
BLEU-1 Score on test data 6662693149: 52.632
BLEU-1 Score on test data 6668564001: 20.000
BLEU-1 Score on test data 6685838035: 61.538
BLEU-1 Score on test data 674513893: 19.786
BLEU-1 Score on test data 675153: 23.077
BLEU-1 Score on test data 6788339456: 38.462
BLEU-1 Score on test data 6800484706: 36.842
BLEU-1 Score on test data 6827875949: 42.737
BLEU-1 Score on test data 6838340009: 36.193
BLEU-1 Score on t

In [13]:
np.mean(scores)

37.679667828529304

In [14]:
print(len(good))
print(len(bad))


144
256
